In [3]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('HPCG.db')

In [1]:
def extract(filename, data):
    with open(filename, 'r') as file:
        lines = file.readlines()

    NX = ""
    NY = ""
    NZ = ""
    T = 1860
    # time is fixed to 1860 now
    Gflops = ""

    idd_index = 10
    # the line index of "Processor Dimensions="
    Gflops_index = 117
    # the line index of "Final Summary="

    NX = lines[idd_index + 1].split["="][1]
    NY = lines[idd_index + 2].split["="][1]
    NZ = lines[idd_index + 3].split["="][1]
    Gflops = lines[Gflops_index + 1].split["="][1]

    data.append(NX, NY, NZ, T, Gflops)
    return data        

In [5]:
import os

# check below before running this script
directory_path = ""  
core_num = 24

file_names = [file for file in os.listdir(directory_path) if file.endswith(".txt")]
data = []

for filename in file_names:
    print(filename)
    extract(directory_path + filename, data)

# conversion
data = [
    [int(value) if index in range(4) else float(value) if index == 4 else value for index, value in enumerate(row)]
    for row in data
]


# 加核数
data = [[core_num] + row for row in data]
df = pd.DataFrame(data, columns=['cores', 'NX', 'NY', 'NZ', 'T', 'GFlops'])
df

N_12_cut.out
N_4_finish.out
N+_54_finish.out
NB_16_finish.out
NB_19_finish.out
NB_4_cut.out
NB_9_finish.out


,cores,PMAP,SWAP,L1,U,EQUIL,DEPTH,BCAST,RFACT,NDIV,PFACT,NBMIN,N,NB,P,Q,Time,Gflops
0,4,Row-major,Mix,transposed,transposed,yes,0,0,R,2,L,2,40704.0,192.0,2.0,2.0,143.75,312.77
1,4,Row-major,Mix,transposed,transposed,yes,0,0,R,2,L,2,45696.0,192.0,2.0,2.0,200.89,316.66
2,4,Row-major,Mix,transposed,transposed,yes,0,0,R,2,L,2,49920.0,192.0,2.0,2.0,279.16,297.10
3,4,Row-major,Mix,transposed,transposed,yes,0,0,R,2,L,2,54144.0,192.0,2.0,2.0,349.04,303.18
4,4,Row-major,Mix,transposed,transposed,yes,0,0,R,2,L,2,57600.0,192.0,2.0,2.0,389.93,326.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,4,Row-major,Mix,transposed,transposed,yes,0,0,R,2,L,2,89088.0,368.0,2.0,2.0,1806.92,260.88
114,4,Row-major,Mix,transposed,transposed,yes,0,0,R,2,L,2,89088.0,376.0,2.0,2.0,1808.94,260.59
115,4,Row-major,Mix,transposed,transposed,yes,0,0,R,2,L,2,89088.0,384.0,2.0,2.0,2199.85,214.28
116,4,Row-major,Mix,transposed,transposed,yes,0,0,R,2,L,2,89088.0,392.0,2.0,2.0,2174.01,216.83


In [6]:
# 存储到数据库
df.to_sql('table1', conn, if_exists='replace', index=False)

118

In [7]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM table1 ORDER BY Gflops DESC LIMIT 1")

results = cursor.fetchall()
results

[(4,
  'Row-major',
  'Mix',
  'transposed',
  'transposed',
  'yes',
  '0',
  '0',
  'R',
  '2',
  'L',
  '2',
  89088.0,
  328.0,
  2.0,
  2.0,
  1294.77,
  364.07)]

In [11]:
# 关闭游标和连接
cursor.close()

# 关闭数据库连接
conn.close()